# Importing in ilcd format

In [1]:
from collections import Counter
from pathlib import Path
import bw2io
import bw2calc
import bw2data
from bw2io.importers.ilcd import ILCDImporter
import pandas as pd

In [2]:
print(bw2io.__version__)

(0, 9, 'DEV10')


In [3]:
gwp100 = ('IPCC 2013', 'climate change', 'GWP 100a')

In [4]:
bw2data.projects.set_current('ilcd_import')
bw2io.bw2setup()

Biosphere database already present!!! No setup is needed


In [5]:
bw2data.databases

Databases dictionary with 9 object(s):
	biosphere3
	example_glad
	example_glad_many
	example_ilcd
	example_ilcd_lcacommons
	lca_commons
	lcacommons_electricity
	lcacommons_forest
	test_db

In [6]:
path_to_examples = Path('bw2io/data/examples/ilcd_examples')
assert path_to_examples.is_dir()
paths_to_examples = list(path_to_examples.glob('*zip'))

## 1) GLAD - single activity

this activity is used for testing purposes. The pdfs have been removed from the zip file to make it lighter

In [7]:
path_to_GLAD_example = Path('bw2io/data/examples/ilcd_example.zip')

In [8]:
so = ILCDImporter(dirpath= path_to_GLAD_example, dbname='example_glad')

In [9]:
so.apply_strategies()
so.match_database('biosphere3',fields=['database','code'])
so.match_database(fields=['database','code'])
so.statistics()

Applying strategy: rename_activity_keys
Applying strategy: get_activity_unit
Applying strategy: set_production_exchange
Applying strategy: convert_to_default_units
Applying strategy: assign_only_product_as_production
Applying strategy: map_to_biosphere3
Applying strategy: set_default_location
Applying strategy: set_activity_parameters
Applied 8 strategies in 0.05 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
1 datasets
15 exchanges
4 unlinked exchanges
  Type biosphere: 2 unique unlinked exchanges
  Type technosphere: 2 unique unlinked exchanges


(1, 15, 4)

There are a number of unlinked exchanges.

### battling with unlinked

In [10]:
unlinked_df = pd.DataFrame(list(so.unlinked))
unlinked_df.loc[unlinked_df['type']=='biosphere',:].T

,0,3
uuid,08a91e70-3ddc-11dd-960b-0050c2490048,fe0acd60-3ddc-11dd-af54-0050c2490048
category_0,Emissions,Emissions
category_1,Emissions to air,Emissions to air
category_2,"Emissions to air, unspecified","Emissions to air, unspecified"
type,biosphere,biosphere
refobj,93a60a56-a3c8-11da-a746-0800200b9a66,93a60a56-a3c8-11da-a746-0800200b9a66
exchanges_internal_id,8,6
exchanges_name,"methane (Emissions to air, unspecified)","carbon dioxide (Emissions to air, unspecified)"
exchanges_direction,Output,Output
amount,0.0,0.022122


First lets try to pass from ilcd 2.0 to ilcd3

In [12]:
ilcd2_ilcd3 = pd.read_excel('ChangeLog_COMPLETE_EF2.0toEF3.0(1).xlsx',sheet_name='Mapped flows',skiprows=2)
ilcd2_ilcd3

,UUID OLD (EF 2.0),name,UUID NEW (EF 3.0),name.1
0,30da22c2-c0b1-467e-a491-3a07d6d72370,HFE-236ca12 (HG-10),d86b9e71-6555-11dd-ad8b-0800200c9a66,HG-10
1,229e91a6-ff71-4eeb-b71b-c66e7fae9215,Argon-40,3f5f02ed-d820-47b1-b319-505ab06b9bc0,Argon-40
2,549c62f1-9693-4da3-814e-d31ca726837a,Argon-40,e8d5b530-d495-4f79-8c34-ca8ec4852bfa,Argon-40
3,041fd22c-6556-11dd-ad8b-0800200c9a66,polonium,cd375ebd-60f8-4260-b1bd-41b2840376cf,polonium
4,73e2c735-8a30-4f18-a846-d7392081a665,Water (evapotranspiration),4b772bba-b5c1-4098-9c1d-3d9d8c74c7fa,Water (evapotranspiration)
...,...,...,...,...
248,c753193c-87e1-4d85-9fa3-61c6662a5302,beta-pinene,041f840a-6556-11dd-ad8b-0800200c9a66,beta-pinene
249,cd46eb6c-8e97-49c3-bb75-19a4483d93b9,beta-pinene,041f840d-6556-11dd-ad8b-0800200c9a66,beta-pinene
250,d1298572-2991-40ff-8c15-8b7141e198e4,beta-pinene,041f8409-6556-11dd-ad8b-0800200c9a66,beta-pinene
251,f089349e-eb83-4343-aa33-96f1e745472c,beta-pinene,fe0acd60-3ddc-11dd-a795-0050c2490048,beta-pinene


In [17]:
ilcd2_ilcd3_dict = ilcd2_ilcd3.set_index('UUID OLD (EF 2.0)')['UUID NEW (EF 3.0)'].to_dict()

In [18]:
unlinked_df.loc[unlinked_df['type']=='biosphere','uuid'].isin(ilcd2_ilcd3_dict)

0    False
3    False
Name: uuid, dtype: bool

bad luck

We can first verify if the UUID's are in the input file https://github.com/UNEP-Economy-Division/GLAD-ElementaryFlowResources/tree/master/Mapping/Input/Flowlists

In [11]:
mapping_input_df = pd.read_csv('ILCD_EFv3.0.csv',low_memory=False)

In [12]:
existing_uuids = mapping_input_df['Flow UUID'].to_list()

In [13]:
unlinked_df.uuid.isin(existing_uuids)

0    False
1    False
2    False
3    False
Name: uuid, dtype: bool

There are not in ilcd 2.0 nor in 3.0. At this point the only solution is a manual mapping

In [15]:
unlinked_df.loc[unlinked_df['type']=='technosphere',:].T

,1,2
uuid,55a4c166-2eb6-43a3-9a13-2e4f2c4fee60,e9fd08fb-8067-4d90-91e3-4c92fc1803f7
category_0,Valuable substances,Others
category_1,Energy carrier,None
category_2,Fuels,None
type,technosphere,technosphere
refobj,93a60a56-a3c8-11da-a746-0800200b9a66,93a60a56-a3c8-11da-a746-0800200b9a66
exchanges_internal_id,1,0
exchanges_name,Diesel,Cargo
exchanges_direction,Input,Input
amount,0.007325,1.0


### write database and examine what is parsed

In [16]:
so.drop_unlinked(True)

Applying strategy: drop_unlinked
Applied 1 strategies in 0.00 seconds


In [17]:
so.write_database()

Title: Writing activities to SQLite3 database:
  Started: 02/15/2023 12:09:35
  Finished: 02/15/2023 12:09:35
  Total time elapsed: 00:00:00
  CPU %: 0.00
  Memory %: 3.05
Created database: example_glad


Brightway2 SQLiteBackend: example_glad

In [19]:
glad_example = bw2data.Database('example_glad').random()

parameters are parsed although not linked to exchanges

In [20]:
pd.DataFrame(glad_example['parameters']) # not yet linked to exchanges

,name,comment,formula,mean,minimum,maximum,std
0,distance,"[km] distance start - end, default = 100 km",share_mw+share_ru+share_ur,100,0.8,1.5,0
1,payload,"[t] default = 1,2 t",0.000179963579284959/(payload*1000*utilisation),1.2,0,2000,0
2,ppm_sulfur,"[ppm] sulphur content in diesel, default Europ...",0.000158765105879866/(payload*1000*utilisation),10,0,1,0
3,share_check,Check - value must be 1,((share_mw*spec_benzene_mw)+(share_ru*spec_ben...,1,0,1,0
4,share_CO2_bio,[-] share of biogenic C in fuel,0.000214096377021633/(payload*1000*utilisation),0.05,0,1,0
5,share_mw,"[-] driving share motorway , default = 0,29",0.00025863031623885/(payload*1000*utilisation),0.29,0,1,0
6,share_ru,"[-] driving share rural , default = 0,26",0.000228165430598892/(payload*1000*utilisation),0.26,0,1,0
7,share_ur,"[-] driving share urban , default = 0,45",((share_mw*spec_CH4_mw)+(share_ru*spec_CH4_ru)...,0.45,None,None,0
8,spec_benzene_mw,[gbenzene/(kg*km)] benzene emission motorway,0.000307683425489813/(payload*1000*utilisation),1.76434881651921E-7,None,None,0
9,spec_benzene_ru,[gbenzene/(kg*km)] benzene emission rural,0.0133990757167339/(payload*1000*utilisation),1.55652064588104E-7,None,None,0


In [21]:
# contacts are available too
pd.DataFrame(glad_example['contacts'])

,email,website,short_name
0,None,None,GaBi user community
1,None,None,thinkstep
2,info@thinkstep.com,http://www.thinkstep.com,GaBi bug forum
3,info@thinkstep.com,http://www.thinkstep.com,GaBi user forum
4,None,None,No official approval by producer or operator
5,gabi@ibp.fraunhofer.de,http://www.ibp.fraunhofer.de/en/Expertise/Life...,IBP-GaBi
6,unep.tie@unep.org,www.lcinitiative.unep.fr,UNEP SETAC Life Cycle Initiative
7,IES-contact@jrc.ec.europa.eu,http://ies.jrc.ec.europa.eu,"European Commission, JRC-IES"
8,info@thinkstep.com,http://www.thinkstep.com,thinkstep
9,gabi@lbp-gabi.de,http://www.LBPGABI.uni-stuttgart.de,IABP-GaBi


In [22]:
glad_example['general_comment']

'- average emission values Euro 3\n- source emissions/driving share: HBEFA 3.1, status January 2010\n- input parameter: distance [km], payload [t], utilisation [-], sulphur content diesel [ppm], driving share motorway/rural/urban\n- average sulphur content: EU = 10 ppm'

In [23]:
glad_example.get('licensetype')

'Other'

In [24]:
glad_example.get('dataset_format')

'ILCD format 1.1'

that may explain the problems with linking.

### test the LCA object

In [25]:
lca = bw2calc.LCA({glad_example:1},gwp100)
lca.lci()
lca.lcia()
lca.score

-0.0008142366941782204

## 2 ) lcacommons - ilcd file with a single process

Another source of ilcd files

from https://www.lcacommons.gov/

In [8]:
# coal extraction and processing - Northern Appalachia, WC, Underground
path_to_example = Path('bw2io/data/examples/ilcd_examples/f29030e5-f681-31ae-9be8-d47f905cb0cb_3281fd83-15b3-4183-997d-dda9ea50dacd.zip')
lcacommons = ILCDImporter(dirpath= path_to_example,dbname='example_ilcd_lcacommons')


In [9]:
lcacommons.apply_strategies()

Applying strategy: rename_activity_keys
Applying strategy: get_activity_unit
Applying strategy: set_production_exchange
Applying strategy: convert_to_default_units
Applying strategy: assign_only_product_as_production
Applying strategy: map_to_biosphere3
Applying strategy: set_default_location
Applying strategy: set_activity_parameters
Applied 8 strategies in 0.03 seconds


In [10]:
lcacommons.match_database('biosphere3',fields=['database','code'])
lcacommons.match_database(fields=['database','code',])
lcacommons.statistics()

Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
1 datasets
2128 exchanges
2127 unlinked exchanges
  Type biosphere: 1135 unique unlinked exchanges
  Type technosphere: 103 unique unlinked exchanges


(1, 2128, 2127)

Instead of using the ilcd to ecoinvent 3.7 mapping, lets use the `FEDEFL to ecoinvent`

In [11]:
path_to_alt_mapping = (Path.cwd() /'bw2io'/'data'/'FEDEFLv1.0.3-ecoinventEFv3.7.xlsx')
assert path_to_alt_mapping.exists()

In [12]:
mapping_df = pd.read_excel(path_to_alt_mapping)
mapping_dict = mapping_df.set_index('SourceFlowUUID')['TargetFlowUUID'].dropna().to_dict()

we can apply this strategy if we have an alternative mapping of uuids to the biosphere3 database

In [13]:
lcacommons.data = bw2io.strategies.ilcd.alternative_map_to_biosphere3(lcacommons.data,
mapping_dict)

In [14]:
lcacommons.match_database('biosphere3',fields=['database','code'])

Applying strategy: link_iterable_by_fields


In [15]:
lcacommons.statistics()

1 datasets
2128 exchanges
1051 unlinked exchanges
  Type biosphere: 752 unique unlinked exchanges
  Type technosphere: 103 unique unlinked exchanges


(1, 2128, 1051)

ok, substantially better. But this dataset probably requires more specific mapping tricks for a better mapping

In [16]:
for i,u in enumerate(lcacommons.unlinked):
    print(f"{u['type']} --- {u['name']} --- {u['uuid']}")

    if i > 10: break

technosphere --- 2,4-DINITROTOLUENE --- d3540cb8-a801-3913-9040-1619ab5b3b46
biosphere --- Methyl parathion --- 0cf75565-4164-3e75-b847-4974ebe979fc
biosphere --- Bicarbonate --- 1208f3d7-8e09-3e47-9fa8-cbef9b54253d
biosphere --- 2,4-D isopropyl ester --- 2fda9070-a8d8-3316-993e-42179ddaf687
biosphere --- Nitroglycerin --- 8ee2febb-ace4-38cc-b478-a62b5cf3ed1d
biosphere --- Benfluralin --- 076a10b9-d926-33f7-b146-10a1e778d9e7
biosphere --- Chromic(VI) acid --- 4717a8b5-2466-3dd1-b856-f639e8442e75
biosphere --- Pyrethrin I --- b61e086a-463f-3b36-b45d-649657392231
biosphere --- Cupric sulfate --- e9be3546-c54f-3ebc-96a2-a8897de5755f
biosphere --- Dimethipin --- 13a4f088-d48e-3c23-bd72-7b74d23a190e
biosphere --- (E)-Crotonic acid --- 9d718e3c-b210-3b88-8d2e-27982bcc69e8
technosphere --- CORROSIVE WASTE --- d5d165f8-aad3-3b82-a849-1d158c5d031d


In [17]:
lcacommons.drop_unlinked(True)

Applying strategy: drop_unlinked
Applied 1 strategies in 0.02 seconds


In [18]:
lcacommons.write_database()

Title: Writing activities to SQLite3 database:
  Started: 02/15/2023 12:22:31
  Finished: 02/15/2023 12:22:31
  Total time elapsed: 00:00:00
  CPU %: 77.40
  Memory %: 5.97
Created database: example_ilcd_lcacommons


Brightway2 SQLiteBackend: example_ilcd_lcacommons

In [19]:
commons_example = bw2data.Database('example_ilcd_lcacommons').random()

In [20]:
surprise = bw2data.Database('example_ilcd_lcacommons').random()
surprise

'coal extraction and processing - Northern Appalachia, WC, Underground' (kilogram, GLO, None)

In [21]:
lca = bw2calc.LCA({surprise:1},gwp100)
lca.lci()
lca.lcia()
lca.score

364.82272957334135

In [30]:
# we stract a wide range of info not always available
surprise.as_dict().keys()

dict_keys(['treatment_standards_routes', 'mix_and_location_types', 'functional_unit_flow_properties', 'uuid', 'general_comment', 'reference_year', 'data_set_valid_until', 'time_representativeness_description', 'location', 'reference_to_reference_flow', 'parameter_name', 'parameter_comment', 'parameter_mean_value', 'parameter_minimum_value', 'parameter_maximum_value', 'parameter_std95', 'parameter_formula', 'intended_application', 'dataset_format', 'licensetype', 'contacts', 'database', 'type', 'code', 'name', 'unit', 'exchanges_name', 'reference product', 'production amount'])

In [31]:
for key in ['general_comment','treatment_standards_routes','mix_and_location_types', 'functional_unit_flow_properties',
'reference_year', 'data_set_valid_until', 'time_representativeness_description', 
'location', 'intended_application', 'dataset_format','licensetype', 'contacts', 
'database', 'type',  'name', 'unit', 'reference product', 'production amount']:
    if surprise.get(key) is not None:
        print('----',key,'\n',surprise.get(key))

---- general_comment 
 The cradle-to-gate inventory for production of coal aggregated to basin, mine type, and coal type groups. For coal extraction there are two major processes that form the basis of the coal life cycle model - underground and surface coal mining. These are connected to auxiliary processes that provide inventories from things like coal mine methane emissions, water use, water emissions, etc. All processes use parameters that allow some differentiation based on region or coal type. Details on the coal modeling can be found in the NETL Coal Baseline report to be published in the near future: netl.doe.gov/LCA This process was created with ElectricityLCI (https://github.com/USEPA/ElectricityLCI) version 1.0.1 using the ELCI_1 configuration.
---- reference_year 
 2016
---- data_set_valid_until 
 2016
---- location 
 GLO
---- intended_application 
 The intended application for these inventories is to provide high-resolution electricity data for the establishment of robust 

## 3) ILCD file with several processes

This dataset has 3 processes

In [34]:
path_to_example = Path('bw2io/data/examples/ilcd_examples/US_Forest_Service_Forest_Products_Lab-Woody_biomass.zip')
lcacommons = ILCDImporter(dirpath= path_to_example,dbname='lcacommons_forest')
lcacommons.apply_strategies()
lcacommons.statistics()

Applying strategy: rename_activity_keys
Applying strategy: get_activity_unit
Applying strategy: set_production_exchange
Applying strategy: convert_to_default_units
Applying strategy: assign_only_product_as_production
Applying strategy: map_to_biosphere3
Applying strategy: set_default_location
Applying strategy: set_activity_parameters
Applied 8 strategies in 0.02 seconds
3 datasets
24 exchanges
24 unlinked exchanges
  Type production: 3 unique unlinked exchanges
  Type technosphere: 16 unique unlinked exchanges


(3, 24, 24)

In [35]:
# link production flows
lcacommons.match_database(fields=['database','code',])
lcacommons.statistics()

Applying strategy: link_iterable_by_fields
3 datasets
24 exchanges
21 unlinked exchanges
  Type technosphere: 16 unique unlinked exchanges


(3, 24, 21)

In [36]:
# link technosphere
lcacommons.match_database(fields=['exchanges_name','unit',],kind='technosphere')
lcacommons.statistics()

Applying strategy: link_iterable_by_fields
3 datasets
24 exchanges
19 unlinked exchanges
  Type technosphere: 15 unique unlinked exchanges


(3, 24, 19)

some examples of unlinked exchanges, that would probably need other datasets from the LCACommons

In [39]:
pd.DataFrame(lcacommons.unlinked).sample(4).T

,14,12,2,8
uuid,f7c00d19-9601-373a-ba23-593e251b4dee,31948a1f-886a-3020-b33b-d381a427dbc6,2afb4e2f-b224-3900-956f-03e3f27b276b,bc38e349-1ccf-3855-a615-a4f581ab875b
category_0,None,None,None,None
category_1,None,None,None,None
category_2,None,None,None,None
type,technosphere,technosphere,technosphere,technosphere
refobj,93a60a56-a3c8-22da-a746-0800200c9a66,93a60a56-a3c8-22da-a746-0800200c9a66,838aaa20-0117-11db-92e3-0800200c9a66,93a60a56-a3c8-22da-a746-0800200c9a66
exchanges_internal_id,1,8,4,9
exchanges_name,"Residual fuel oil, at refinery","Pulpwood, softwood, national forest, steep slo...","Transport, combination truck, average fuel mix","Gasoline, combusted in equipment"
exchanges_direction,Input,Input,Input,Input
amount,0.000361,0.01344,0.000294,0.000639


In [40]:
lcacommons.drop_unlinked(True)

Applying strategy: drop_unlinked
Applied 1 strategies in 0.00 seconds


In [41]:
lcacommons.write_database()

Title: Writing activities to SQLite3 database:
  Started: 02/15/2023 12:39:52
  Finished: 02/15/2023 12:39:52
  Total time elapsed: 00:00:00
  CPU %: 0.00
  Memory %: 5.97
Created database: lcacommons_forest


Brightway2 SQLiteBackend: lcacommons_forest

In [47]:
len(bw2data.Database('lcacommons_forest'))

3

In [43]:
act = bw2data.Database('lcacommons_forest').random()
act

'Wood chip briquette production; at plant' (kilogram, US, None)

In [44]:
lca =  bw2calc.LCA({act:1})

In [45]:
lca.lci()

/home/migmig/Documents/repos/brightway2-io/venv_ilcd/lib/python3.9/site-packages/bw2calc/lca.py:228: UserWarning: No biosphere flows found. No inventory results can be calculated, `lcia` will raise an error
  warnings.warn("No biosphere flows found. No inventory results can "


In [46]:
lca.technosphere_matrix.todense()

matrix([[ 0.0436,  0.    ,  0.    ],
        [ 0.    ,  1.    ,  0.    ],
        [-0.063 , -1.    ,  1.    ]])

The 3 processes in the ilcd file are linked, but connexion to other datasets within the LCAcommons is lost.